In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('ds').getOrCreate()

In [2]:
df = spark.sql('select * from college')

In [3]:
df.show(5)

+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+
 School|Private|Apps|Accept|Enroll|Top10perc|Top25perc|F_Undergrad|P_Undergrad|Outstate|Room_Board|Books|Personal|PhD|Terminal|S_F_Ratio|perc_alumni|Expend|Grad_Rate|
+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+
Abilene Christian...| Yes|1660| 1232| 721| 23| 52| 2885| 537| 7440| 3300| 450| 2200| 70| 78| 18.1| 12| 7041| 60|
 Adelphi University| Yes|2186| 1924| 512| 16| 29| 2683| 1227| 12280| 6450| 750| 1500| 29| 30| 12.2| 16| 10527| 56|
 Adrian College| Yes|1428| 1097| 336| 22| 50| 1036| 99| 11250| 3750| 400| 1165| 53| 66| 12.9| 30| 8735| 54|
 Agnes Scott College| Yes| 417| 349| 137| 60| 89| 510| 63| 12960| 5450| 450| 875| 92| 97| 7.7| 37| 19016| 59|
Alaska Pacific Un...| Yes| 193| 146| 55| 16| 44| 249| 869| 7560| 4120| 800| 1500| 76| 72| 11.9| 2| 10922| 15|
+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+
only showing top 5 rows

In [4]:
df.printSchema()

root
-- School: string (nullable = true)
-- Private: string (nullable = true)
-- Apps: integer (nullable = true)
-- Accept: integer (nullable = true)
-- Enroll: integer (nullable = true)
-- Top10perc: integer (nullable = true)
-- Top25perc: integer (nullable = true)
-- F_Undergrad: integer (nullable = true)
-- P_Undergrad: integer (nullable = true)
-- Outstate: integer (nullable = true)
-- Room_Board: integer (nullable = true)
-- Books: integer (nullable = true)
-- Personal: integer (nullable = true)
-- PhD: integer (nullable = true)
-- Terminal: integer (nullable = true)
-- S_F_Ratio: double (nullable = true)
-- perc_alumni: integer (nullable = true)
-- Expend: integer (nullable = true)
-- Grad_Rate: integer (nullable = true)

In [5]:
df.groupBy('Private').count().show()

+-------+-----+
Private|count|
+-------+-----+
 No| 212|
 Yes| 565|
+-------+-----+

In [6]:
from pyspark.ml.feature import VectorAssembler

In [7]:
df.columns

Out[ 8 ]: 
['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [8]:
assembler = VectorAssembler(inputCols=['Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate'],outputCol='features')

In [9]:
output = assembler.transform(df)

In [10]:
output.show(5)

+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+--------------------+
 School|Private|Apps|Accept|Enroll|Top10perc|Top25perc|F_Undergrad|P_Undergrad|Outstate|Room_Board|Books|Personal|PhD|Terminal|S_F_Ratio|perc_alumni|Expend|Grad_Rate| features|
+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+--------------------+
Abilene Christian...| Yes|1660| 1232| 721| 23| 52| 2885| 537| 7440| 3300| 450| 2200| 70| 78| 18.1| 12| 7041| 60|[1660.0,1232.0,72...|
 Adelphi University| Yes|2186| 1924| 512| 16| 29| 2683| 1227| 12280| 6450| 750| 1500| 29| 30| 12.2| 16| 10527| 56|[2186.0,1924.0,51...|
 Adrian College| Yes|1428| 1097| 336| 22| 50| 1036| 99| 11250| 3750| 400| 1165| 53| 66| 12.9| 30| 8735| 54|[1428.0,1097.0,33...|
 Agnes Scott College| Yes| 417| 349| 137| 60| 89| 510| 63| 12960| 5450| 450| 875| 92| 97| 7.7| 37| 19016| 59|[417.0,349.0,137....|
Alaska Pacific Un...| Yes| 193| 146| 55| 16| 44| 249| 869| 7560| 4120| 800| 1500| 76| 72| 11.9| 2| 10922| 15|[193.0,146.0,55.0...|
+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+--------------------+
only showing top 5 rows

In [11]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol='Private',outputCol='PrivateIndex')
indexed_data = indexer.fit(output).transform(output)

In [12]:
final_data = indexed_data.select('features','PrivateIndex')

In [13]:
final_data.show(5)

+--------------------+------------+
 features|PrivateIndex|
+--------------------+------------+
[1660.0,1232.0,72...| 0.0|
[2186.0,1924.0,51...| 0.0|
[1428.0,1097.0,33...| 0.0|
[417.0,349.0,137....| 0.0|
[193.0,146.0,55.0...| 0.0|
+--------------------+------------+
only showing top 5 rows

In [14]:
train,test = final_data.randomSplit([0.7,0.3])

In [15]:
from pyspark.ml.classification import DecisionTreeClassifier,RandomForestClassifier,GBTClassifier
from pyspark.ml import Pipeline

In [16]:
dtc = DecisionTreeClassifier(featuresCol='features',labelCol='PrivateIndex',predictionCol='prediction')
rfc = RandomForestClassifier(featuresCol='features',labelCol='PrivateIndex',predictionCol='prediction',numTrees=100)
gbtc = GBTClassifier(featuresCol='features',labelCol='PrivateIndex',predictionCol='prediction')

In [17]:
# fitting the models
dtc_model =dtc.fit(train)
rfc_model = rfc.fit(train)
gbtc_model = gbtc.fit(train)

In [18]:
# predictions
dtc_pred = dtc_model.transform(test)
rfc_pred = rfc_model.transform(test)
gbtc_pred = gbtc_model.transform(test)

In [19]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
acc_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', 
                                         labelCol='PrivateIndex', 
                                         metricName='areaUnderROC')

In [20]:
dtc_eval= acc_eval.evaluate(dtc_pred)
rfc_eval= acc_eval.evaluate(rfc_pred)
gbtc_eval= acc_eval.evaluate(gbtc_pred)

In [21]:
print('DTC AUC')
print(dtc_eval)

DTC AUC
0.88956154834

In [22]:
print('RFC AUC')
print(rfc_eval)

RFC AUC
0.916712529811

In [23]:
print('GBTC AUC')
print(gbtc_eval)

GBTC AUC
0.895890662264